<a href="https://colab.research.google.com/github/akash02ita/CPSC-599.6-DL-for-Vision/blob/proj/proj/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --branch proj https://github.com/akash02ita/CPSC-599.6-DL-for-Vision.git myrepo
!cp myrepo/proj/CustomDataset.ipynb .

Cloning into 'myrepo'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 101 (delta 12), reused 23 (delta 0), pack-reused 0
Receiving objects: 100% (101/101), 17.53 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
__BACKUP__NAME__ = __name__
__name__ = "NOT MAIN" # import jupyter methods without running test scripts
%run CustomDataset.ipynb
__name__ = __BACKUP__NAME__

# Setup Custom Dataset

In [3]:
from google.colab import drive
import shutil, zipfile

if __name__ == "__main__":
  # Mount the folder
  drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Setup handwritten text database

In [4]:
DATASET_PATH = "/content/drive/MyDrive/599 Project/custom-dataset-script"

handwritten_zip_path = "/content/drive/MyDrive/599 Project/custom-dataset-script/handwritten-text-images-database.zip"
extract_zip_to(handwritten_zip_path, ".")

'.'

In [5]:
# 'en_test' folder not present.
# train_handwritten = TextDataset("database/en_train_filtered", "database/en_train_filtered/labels.csv")
# val_handwritten = TextDataset("database/en_val", "database/en_val/labels.csv")

# split the texts into training, validation and test (60, 20, 20)
handwritten_dataset = TextDataset("database/en_train_filtered", "database/en_train_filtered/labels.csv")
from torch.utils.data import random_split

train_size = int(0.6 * len(handwritten_dataset))
val_size = int(0.2 * len(handwritten_dataset))
test_size = len(handwritten_dataset) - train_size - val_size

train_handwritten, val_handwritten, test_handwritten = random_split(handwritten_dataset, [train_size, val_size, test_size])
print(len(train_handwritten),len(val_handwritten),len(test_handwritten))

11999 3999 4001


## Setup printed text database

In [6]:
"""
!pip install english-words
from english_words import get_english_words_set
!pip install essential_generators
from essential_generators import DocumentGenerator
"""
!pip install nltk
import nltk
nltk.download('words')
from nltk.corpus import words
from random import sample

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [7]:
# https://stackoverflow.com/questions/55200307/generate-a-string-of-n-random-english-words-with-nltk-python
def random_sentence(n):
    return ' '.join(sample(words.words(), n)) # calling it too many times is  inefficient
    # maybe better to use a prexisting word database (althougt not neeeded as sentences.txt has already been generated, and only once is required)

In [8]:
import random, shutil
# GENERATE_SENTENCES = input("Do you want to generate sentences? (y/n) ") == "y"
GENERATE_SENTENCES = False # already generated before. Hardcode to false to be safe
PRINTED_TEXTS_PATH = DATASET_PATH # "/content/drive/MyDrive/599 Project/custom-dataset-script"
class Sentence:
  def __init__(self, tot_sentences=1, min_length=1, max_length=3):
    assert 1 <= min_length <= max_length
    # initialize sentences
    self.tot_sentences = tot_sentences
    self.sentences = [""]*tot_sentences
    for i in range(tot_sentences):
      sentence_length = random.randint(min_length,max_length)
      self.sentences[i] = random_sentence(sentence_length)
  def __len__(self):
    return self.tot_sentences
  def __getitem__(self, i):
    return self.sentences[i]
  def __str__(self):
    return str(self.sentences)

if GENERATE_SENTENCES:
  texts = Sentence(10000)
  # texts = Sentence(100)
  print("total sentences:", len(texts))
  # print(texts)

  # export to file
  with open('sentences.txt', 'w') as file:
    for sentence in texts:
      file.write(sentence+"\n")
  # copy file to drive
  shutil.copy("sentences.txt", PRINTED_TEXTS_PATH)

# copy file from drive
shutil.copy(os.path.join(DATASET_PATH, "sentences.txt"),".")

'./sentences.txt'

In [9]:
with open('sentences.txt', 'r') as file:
  texts = list(map(lambda t: t.rstrip('\n'), file.readlines()))

# split the texts into training, validation and test (60, 20, 20)
from torch.utils.data import random_split

train_size = int(0.6 * len(texts))
val_size = int(0.2 * len(texts))
test_size = len(texts) - train_size - val_size

sentences = texts
print(len(sentences))

10000


In [10]:
# setup one font
extract_zip_to("/content/drive/MyDrive/599 Project/custom-dataset-script/fonts/roboto.zip", "./fonts")
FONT_PATH =  "./fonts/Roboto-Regular.ttf"

# GENERATE_PRINTED_IMAGES = input("Do you want to generate printed images? (y/n) ") == "y"
GENERATE_PRINTED_IMAGES = False # from now and on, to directly extract instead of re-generating

import shutil, os
def zip_to(src, dest):
  # create zip file with the nested folder as the folder `src`
  # so that when extracting `src` folder is extracted instead of `src/*` files
  shutil.make_archive(dest, 'zip', ".", src)


if GENERATE_PRINTED_IMAGES:
  # generate printed images
  generatePrintedImages("printed_text_dataset", FONT_PATH, sentences)

  # zip and save all them on drive
  zip_to("printed_text_dataset", os.path.join(DATASET_PATH, "printed_text_dataset"))  

# extract zip files from drive 
print(os.listdir(DATASET_PATH))
extract_zip_to(os.path.join(DATASET_PATH, "printed_text_dataset.zip"), ".")
print(os.listdir(DATASET_PATH))

Do you want to generate printed images? (y/n) y
['handwritten-text-images-database.zip', 'handwritten-text-images-labels.csv', 'sample images', 'fonts', 'Custom Dataset.ipynb', 'sentences.txt', 'train_text_printed.zip', 'val_text_printed.zip', 'test_text_printed.zip', 'printed_text_dataset.zip']
['handwritten-text-images-database.zip', 'handwritten-text-images-labels.csv', 'sample images', 'fonts', 'Custom Dataset.ipynb', 'sentences.txt', 'train_text_printed.zip', 'val_text_printed.zip', 'test_text_printed.zip', 'printed_text_dataset.zip']


In [12]:
# split the texts into training, validation and test (60, 20, 20)
printed_dataset = TextDataset("printed_text_dataset", "printed_text_dataset/labels.csv")
from torch.utils.data import random_split

train_size = int(0.6 * len(printed_dataset))
val_size = int(0.2 * len(printed_dataset))
test_size = len(printed_dataset) - train_size - val_size

train_printed, val_printed, test_printed = random_split(printed_dataset, [train_size, val_size, test_size])
print(len(train_printed),len(val_printed),len(test_printed))

24000 8000 8000


In [16]:
!ls printed_text_dataset/ | wc -l
print(val_printed[0][1])
val_printed[0][0].show()
print(val_printed[1][1])
val_printed[1][0].show()
print(val_printed[2][1])
val_printed[2][0].show()
print(val_printed[3][1])
val_printed[3][0].show()

40001
feudal


gascoigny entoptoscope


unripping


alvus


## Create custom dataset

In [32]:
# get orignal images: https://www.kaggle.com/datasets/plameneduardo/sarscov2-ctscan-dataset?resource=download
extract_zip_to(os.path.join(DATASET_PATH, "covid_ctscan_dataset.zip"),".")
!mv COVID "originaldataset" || rm -r "original_images"; mv COVID "original_images" 
!rm -r non-COVID

mv: cannot stat 'COVID': No such file or directory


In [37]:
# in custom dataset assertion error is caused by fact that random_split creates new instance type
print(type(train_printed))
train_customdataset = CustomDataset("original_images", train_printed, train_handwritten)
val_customdataset = CustomDataset("original_images", val_printed, val_handwritten)
test_customdataset = CustomDataset("original_images", test_printed, test_handwritten)

# easy and better solution
"""
pass whole text dataset and handwritten dataset to customdataset

then use randomsplit after it

not only it is easier and avoid assertion error but it is also better:
- custom dataset, may generate of whatever length
- in the previous case it would (if assertion error does not occur) length rations between train, val, test are thus 'undeterministic'
- it allows to mantain ration of lengths between train, val, test, if it splitted at the end stage
"""

<class 'torch.utils.data.dataset.Subset'>


AssertionError: ignored

# Model

## model setup

## model training

## model evaluation